# Want to collect your data from the web and store it easily ?

In this Notebook we will create a Data Base in mySQL , parse COT report from CFTC and store it with report date, in 3 steps :

### A. Create mySQL Table
### B. Parse CFTC website with information required from the Commitment of Trader report, released every Tuesday and Friday
### C. Store in DB using SQL query


# -------------------------------------

#### Some things you should consider before web scraping a website:

1.) You should check a site's terms and conditions before you scrape them. 

2.) Space out your requests so you don't overload the site's server, doing this could get you blocked.

3.) Scrapers break after time - web pages change their layout all the time, you'll more than likely have to rewrite your code. 

4.) Web pages are usually inconsistent, more than likely you'll have to clean up the data after scraping it.

5.) Every web page and situation is different, you'll have to spend time configuring your scraper.



#### There are three modules we'll need in addition to python are:

1.) BeautifulSoup, which you can download by typing: *pip install beautifulsoup4* or *conda install beautifulsoup4* (for the Anaconda distrbution of Python) in your command prompt.

2.) lxml , which you can download by typing: *pip install lxml* or *conda install lxml* (for the Anaconda distrbution of Python) in your command prompt.



We'll start with our imports:

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
from pandas import Series,DataFrame

In [6]:
# Request content from web page
result = requests.get(url)
c = result.content

# Set as Beautiful Soup Object
soup = BeautifulSoup(c, "lxml")

In [4]:
from urllib import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re
import csv
import datetime

url = 'http://www.cftc.gov/dea/options/petroleumlof.htm'
response = urllib2.urlopen(url)
cr = csv.reader(response)

## A. Create DB to store value of positions of Money Managers in the Crude Oil Futures market


In [ ]:
# We create the Data Base in mySQL with pymsql

conn = pymysql.connect(host='127.0.0.1', user='Baptiste',passwd='Kamca2017!', db='scraping')
cur = conn.cursor()
cur.execute("USE scraping")



#On cree la table Brent 

cur.execute("CREATE TABLE Volume_OI ( MDATE date NOT NULL,\
                                        FUT_AGGTE_VOL_BRENT FLOAT(15,2),\
                                        BRENT_1 FLOAT(15,2),\
                                        BRENT_OI FLOAT(15,2),\
                                        WTI_OI FLOAT(15,2),\
                                        WTI_Managed_Money_Net FLOAT(15,2),   \
                                        BRENT_Managed_Money_Net FLOAT(15,2),\
                                        WTI_Managed_Money_Long FLOAT(15,2), BRENT_Managed_Money_Long FLOAT(15,2),WTI_Managed_Money_Short FLOAT(15,2),BRENT_Managed_Money_Short FLOAT(15,2),created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, \
                                        PRIMARY KEY(MDATE));")

conn.commit()
cur.close()
conn.close()

## B. Parse information with Beautifulsoup and Regex python librairies

In [3]:
# On fait le test pour extraire la donnee que l'on cherche du rapport du cftc
import csv
import urllib2
import re
from urllib2 import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame

html = urlopen("http://www.cftc.gov/dea/options/petroleum_lof.htm")
bsObj = BeautifulSoup(html.read())
text = []
index = 0
#test = bsObj.find("pre",re.compile("[Code-02165E]+"))
test2 = bsObj.findAll("pre")

for item in test2:
    if 'txt' in item:
        # Add the date and reports
        text.append(test2[index-1])
    index += 1
    
test = bsObj.prettify()

line_re = re.compile('(Code-02165E)\w+')

#records = line_re.find(test)
#for item in test2[0]:
#    print item
#records

mysrch = re.search('(Code-067651)+',test)
mysrch2 = re.search('(Code-067657)+',test)

tableNYMEX = test[mysrch.start():mysrch2.start()]

entries = re.split("\n+", tableNYMEX)


#On va mettre les donnees dans une nouvelle list en format float pour etre ensuite integrees dans la BDD
sep = re.sub(" ",";",entries[10])
sep = re.sub(":",";",sep)
sep = re.split(';',sep)
sep2 = []
for item in sep:

    if item not in [" ","","u''",None,"All"]:
        sep2.append(float(re.sub(",","",item)))
        
# Il faut recuperer la date a laquelle les positions sont entrees
import dateutil.parser as dparser
import datetime
date = dparser.parse(entries[1],fuzzy=True)

date = date.strftime('%Y-%m-%d')


dtemp =[]

for i in range(10):
    if i==0:
        dtemp.append(date)
    if i==6:
        dtemp.append(sep2[6])
    elif i==8:
        dtemp.append(sep2[7])
    elif i==4:
        dtemp.append(sep2[6]-sep2[7])
    else:
        dtemp.append(None)
        
print dtemp

['2017-12-05', None, None, None, None, 392431.0, None, 439751.0, None, 47320.0, None]


## C. Let's store the list in our DB using SQL code

In [4]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='Baptiste',passwd='Kamca2016!', db='scraping')
cur = conn.cursor()
cur.execute("USE scraping")

#On rentre les donnees dans la BDD
sql = 'INSERT INTO volume_oi (MDATE,FUT_AGGTE_VOL_BRENT,BRENT_1,BRENT_OI,WTI_OI,WTI_Managed_Money_Net,BRENT_Managed_Money_Net,WTI_Managed_Money_Long,BRENT_Managed_Money_Long,WTI_Managed_Money_Short,BRENT_Managed_Money_Short) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)\
                          ON DUPLICATE KEY UPDATE MDATE=VALUES(MDATE);'

cur.execute(sql,dtemp)    
    
conn.commit()
cur.close()
conn.close()

#### Conclusion:

- We need to run the query everyweek as link given is for latest report only
- We illustrate information in a separate notebook